In [1]:
import os
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

from google.colab import files
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Deklarasi path untuk setiap kelas
path_data = '/content/drive/MyDrive/Dataset/masker'
path_withMask = os.path.join(path_data, 'with_mask')
path_withOutMask = os.path.join(path_data, 'without_mask')
path_incorrect = os.path.join(path_data, 'mask_weared_incorrect')

In [ ]:
#menyimpan list nama file data
train_withMask_fnames = os.listdir(path_withMask)
train_withOutMask_fnames = os.listdir(path_withOutMask)
train_incorrect_fnames = os.listdir(path_incorrect)

print(train_withMask_fnames[:5])
print(train_withOutMask_fnames[:5])
print(train_incorrect_fnames[:5])

In [ ]:
print('total gambar yang memakai masker :', len(os.listdir(path_withMask)))
print('total gambar yang tidak memakai masker :', len(os.listdir(path_withOutMask)))
print('total gambar yang memakai masker dengan cara yang salah :', len(os.listdir(path_incorrect)))

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# untuk menampilkan gambar dalam bentuk 6x3
nrows = 6
ncols = 3

pic_index = 0 # Index for iterating over images

fig = plt.gcf()
fig.set_size_inches(ncols*2, nrows*2)

pic_index+=3

next_withMask_pix = [os.path.join(path_withMask, fname) 
                for fname in train_withMask_fnames[ pic_index-3:pic_index] 
               ]

next_withOutMask_pix = [os.path.join(path_withOutMask, fname) 
                for fname in train_withOutMask_fnames[ pic_index-3:pic_index]
               ]

next_incorrect_pix = [os.path.join(path_incorrect, fname) 
                for fname in train_incorrect_fnames[ pic_index-3:pic_index]
               ]

for i, img_path in enumerate(next_withMask_pix+next_withOutMask_pix+next_incorrect_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#Persiapan sebelum splitting data
#Membuat folder untuk train dan validasi setiap kelas

path_data_baru = './Dataset'
os.mkdir(path_data_baru)

train_dir = os.path.join(path_data_baru,'train')
test_dir = os.path.join(path_data_baru,'val')

withMask_train = os.path.join(train_dir,'withMask')
withMask_test = os.path.join(test_dir,'withMask')

withOutMask_train = os.path.join(train_dir,'withOutMask')
withOutMask_test = os.path.join(test_dir,'withOutMask')

incorrect_train = os.path.join(train_dir,'incorrect')
incorrect_test = os.path.join(test_dir,'incorrect')

list_dir = [train_dir,test_dir, withMask_train, withMask_test,
            withOutMask_train, withOutMask_test, incorrect_train, incorrect_test]

for i in list_dir:
  os.mkdir(i)

In [ ]:
#Data splitting dengan validation set = 10% total data
from sklearn.model_selection import train_test_split

train_withMask_dir, test_withMask_dir = train_test_split(os.listdir(path_withMask), test_size = 0.10)
train_withOutMask_dir, test_withOutMask_dir = train_test_split(os.listdir(path_withOutMask), test_size = 0.10)
train_incorrect_dir, test_incorrect_dir = train_test_split(os.listdir(path_incorrect), test_size = 0.10)

In [ ]:
#Memindahkan hasil split ke folder yang telah dibuat
for i in train_withMask_dir:
  shutil.copy(os.path.join(path_withMask,i), os.path.join(withMask_train,i))
for i in test_withMask_dir:
  shutil.copy(os.path.join(path_withMask,i), os.path.join(withMask_test,i))

for i in train_withOutMask_dir:
  shutil.copy(os.path.join(path_withOutMask,i), os.path.join(withOutMask_train,i))
for i in test_withOutMask_dir:
  shutil.copy(os.path.join(path_withOutMask,i), os.path.join(withOutMask_test,i))

for i in train_incorrect_dir:
  shutil.copy(os.path.join(path_incorrect,i), os.path.join(incorrect_train,i))
for i in test_incorrect_dir:
  shutil.copy(os.path.join(path_incorrect,i), os.path.join(incorrect_test,i))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    vertical_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    vertical_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size =(150,150),
    batch_size = 32,
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical')

In [ ]:
#Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

model = Sequential()

#Konvolusi 1 + Input Layer 1
model.add(Conv2D(16, (3,3), activation='relu',
                 input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2,2))
#Konvolusi 2
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
#Konvolusi 3
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
#Konvolusi 4
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
#Flatten
model.add(Flatten())
#Hidden Layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
#Hidden Layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
#Output Layer
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

In [ ]:
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.95):
      self.model.stop_training = True

stopTraining = Callback()

In [ ]:
history = model.fit(train_generator,
                    validation_data=test_generator,
                    epochs=1000,
                    callbacks=[stopTraining])

In [ ]:
plt.plot(history.history['accuracy'], label='Training', color='blue')
plt.plot(history.history['val_accuracy'], label='Validation', color='red')
plt.title('Accuracy Training & Validation')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training', color='blue')
plt.plot(history.history['val_loss'], label='Validation', color='red')
plt.title('Loss Training & Validation')
plt.xlabel('Epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
def predict(image_upload):
  for fn in image_upload.keys():
    # predicting images
    path = fn
    img = image.load_img(path, target_size=(150,150))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
 
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
  
    print(fn)
    if classes[0,0] != 0:
      print('Tidak Memakai Masker')
    elif classes[0,1] != 0:
      print('Memakai Masker')
    else:
      print('Memakai Masker Dengan Cara yang Salah')

In [ ]:
image_test = files.upload()
predict(image_test)